In [63]:
import torch
from torch import nn
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from sklearn.metrics import euclidean_distances
from cave.cavefacade import CAVE
from ConfigSpace.util import impute_inactive_values
from ConfigSpace import CategoricalHyperparameter, UniformFloatHyperparameter, UniformIntegerHyperparameter, OrdinalHyperparameter
from cave.utils.helpers import combine_runhistories
from cave.utils.helpers import create_random_runhistories, combine_random_local, create_new_rhs  # Julia BA
import torchvision
from torch.autograd import Variable

import warnings
import pickle
import collections

In [61]:
cave = CAVE(folders=["../DataForTesting/cplex_regions200/SMAC3/run-1/smac3-output_2019-03-15_09:55:14_185212/run_1/"],
            output_dir="./CAVE/Output/June/AutoEncoder/03_06_CAVE_cplex_jupyter_autoencoder/",
            ta_exec_dir=["../DataForTesting/cplex_regions200/SMAC3/run-1/"],
            file_format='SMAC3',
            # verbose_level='DEBUG'
           )

print("Finished")

Finished


In [68]:
scenario = cave.scenario
from copy import deepcopy
configspace = deepcopy(scenario.cs)

runhistory = cave.global_original_rh
training, transform = create_random_runhistories(runhistory)

dicti = configspace._hyperparameters.items()

In [ ]:
training.save_json('training.json')
import pickle

pickle.dump(training, open("training.pkl", 'wb'), protocol=0)
pickle.load(open("training.pkl", 'rb'))

### Review Configspace

In [ ]:
configspace.get_hyperparameters()                       # List of all Hyperparameter with name, type, choices/interval
configspace.get_hyperparameters()[0]                    # Index return hyperparamter at this place
configspace.get_hyperparameters()[0].name               # Specification what is needed of this hyperparameter
configspace.get_hyperparameter_names()                  # List of the names of all hyperparameter

training.get_all_configs_combined()                     # list of all configurations
training.get_all_configs_combined()[0]                  # Returns the configuration at the place of index
name = configspace.get_hyperparameters()[0].name
training.get_all_configs_combined()[0].get(name)        # Get value of the configuration of the defined hyperparameter

# Convert Data


## Case 1

* standardize continual Data
* replace nan with -1

In [69]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

for hp in configspace.get_hyperparameters():
    if type(hp) is CategoricalHyperparameter:
        hp.choices = hp.choices + (-1234, )

    
values = [OneHotEncoder(categories='auto').fit((np.sort(np.array(hp.choices)).reshape((-1,1))))
          if type(hp) is CategoricalHyperparameter 
          else (StandardScaler().fit(np.array([confi.get(hp.name) for confi in training.get_all_configs_combined()]).reshape(-1, 1))
                 if type(hp) in {UniformFloatHyperparameter, UniformIntegerHyperparameter, OrdinalHyperparameter}
                 else None)
            for hp in configspace.get_hyperparameters()]

In [55]:
for i in range(len(values)):
    if type(values[i]) == StandardScaler and type(values[i]) != OneHotEncoder:
        pass
    elif type(values[i]) == OneHotEncoder and type(values[i]) != StandardScaler:
        pass
    else:
        print("Fehler")

In [70]:
config = training.get_all_configs_combined()[0]
for hp in configspace.get_hyperparameters():
    if type(hp) is CategoricalHyperparameter:
        print(hp.name, hp.choices)
        
# print(config)
# print(hp)
# OneHotEncoder(categories='auto').fit(np.vstack((np.sort(np.array(hp.choices)).reshape((-1,1)), [[-1]])))
#one_hot_encode(training.get_all_configs_combined()[0])

barrier_algorithm ('0', '1', '2', '3', -1234)
barrier_crossover ('-1', '0', '1', '2', -1234)
barrier_limits_corrections ('-1', '0', '1', '4', '16', '64', -1234)
barrier_ordering ('0', '1', '2', '3', -1234)
barrier_startalg ('1', '2', '3', '4', -1234)
emphasis_memory ('no', -1234)
emphasis_mip ('0', '1', '2', '3', '4', -1234)
emphasis_numerical ('yes', 'no', -1234)
feasopt_mode ('0', '1', '2', '3', '4', '5', -1234)
lpmethod ('0', '1', '2', '3', '4', '5', '6', -1234)
mip_cuts_cliques ('-1', '0', '1', '2', '3', -1234)
mip_cuts_covers ('-1', '0', '1', '2', '3', -1234)
mip_cuts_disjunctive ('-1', '0', '1', '2', '3', -1234)
mip_cuts_flowcovers ('-1', '0', '1', '2', -1234)
mip_cuts_gomory ('-1', '0', '1', '2', -1234)
mip_cuts_gubcovers ('-1', '0', '1', '2', -1234)
mip_cuts_implied ('-1', '0', '1', '2', -1234)
mip_cuts_mcfcut ('-1', '0', '1', '2', -1234)
mip_cuts_mircut ('-1', '0', '1', '2', -1234)
mip_cuts_pathcut ('-1', '0', '1', '2', -1234)
mip_cuts_zerohalfcut ('-1', '0', '1', '2', -1234)


In [71]:
# one hot encoding
def one_hot_encode(config):
    # Code from PhMueller
    
    # Create array with one hot encoded values
    result_vec = np.array([]).reshape((-1, 1))  # .astype(object)

    for i, hp in enumerate(configspace.get_hyperparameters()):
        
        val = np.array(config.get(hp.name)).reshape((-1, 1))
        # print(val)
        # case if categorical
        if type(values[i]) is OneHotEncoder:
            if val == [[None]]:
                # val = np.array(['-1']).reshape((-1, 1))
                val = np.array([['-1234']])
            if len(result_vec) == 0:
                result_vec = values[i].transform(val).toarray()  # .astype(object)
            else:
                result_vec = np.hstack((result_vec, 
                                        values[i].transform(val).toarray()))

        # if it is continous
        else:
            if val == [[None]]:
                if len(result_vec) == 0:
                    result_vec = np.array([-1234]).reshape((-1, 1))
                else:
                    result_vec = np.hstack((result_vec, [[-1234]]))
            elif len(result_vec) == 0:
                result_vec = values[i].transform(val)
            else:
                result_vec = np.hstack((result_vec, 
                                        values[i].transform(val)))
        
    return result_vec

In [ ]:
for i in range(len(values)):
    if i == None:
        print("Error")

In [72]:
convert_data = np.array([]).reshape((-1, 1))

for confi in range(len(training.config_ids)):
    if confi % 500 == 0:
        print(confi)
    if len(convert_data) == 0:
        convert_data = one_hot_encode(training.get_all_configs_combined()[confi])
        continue
    convert_data = np.vstack((convert_data, one_hot_encode(training.get_all_configs_combined()[confi])))

print(len(convert_data))
# [one_hot_encode(training.get_all_configs_combined()[confi]) for confi in range(len(training.config_ids))]

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5353


In [ ]:
convert_data_transform = np.array([]).reshape((-1, 1))

for confi in range(len(transform.config_ids)):
    if confi % 10 == 0:
        print(confi)
    if len(convert_data_transform) == 0:
        convert_data_transform = one_hot_encode(transform.get_all_configs_combined()[confi])
        continue
    convert_data_transform = np.vstack((convert_data_transform, one_hot_encode(transform.get_all_configs_combined()[confi])))

print(len(convert_data_transform))

In [ ]:
convert_data.shape[1] == convert_data_transform.shape[1]

In [73]:
np.save("convert_data.npy", convert_data)

In [74]:
convert_data.shape

(5353, 349)

In [ ]:
np.load("convert_data.npy")

# AutoEncoder

In [ ]:
class Softmax_BA(nn.Module):
    """My own class with softmax and crossentropy to transform tensor back in original strucutre"""
    __constants__ = ['dim']

    def __init__(self, num_category, transform_list, confi, dim=None):
        super(Softmax_BA, self).__init__()
        self.num_cat = num_category
        self.transform_list = transform_list
        self.configspace = confi
        self.dim = dim

    def forward(self, x):
        indexing = 0
        x_ = x.clone()
        softmax = nn.Softmax(dim=1)
        for num in range(len(self.configspace.get_hyperparameters())):
            if type(self.transform_list[num]) == OneHotEncoder:
                x_[:, indexing:indexing+self.num_cat[num]] = softmax(x[:, indexing:indexing+self.num_cat[num]])
                indexing += self.num_cat[num]
            else:
                indexing += 1
        x = x_
        return x# Variable(x.data, requires_grad=True)        

In [ ]:
class Autoencoder(nn.Module):
    """ Our autoencoder class. """
    
    def __init__(self, length, act_f, num_layers):
        super(Autoencoder, self).__init__()
        
        if act_f.lower() == 'relu':
            self.act_f = torch.nn.ReLU()
        else:
            self.act_f = torch.nn.Tanh()
        
        assert num_layers > 1
        
        self.encoder_layer = nn.ModuleList(
            [nn.Linear(int(length/(i+1)), int(length/(i+2))) for i in range(num_layers-1)]
        )
        self.encoder_layer.extend([nn.Linear(int(length/(num_layers)), 2)])

        self.decoder_layer = nn.ModuleList(
            [nn.Linear(2, int(length/(num_layers)))]
        )
        self.decoder_layer.extend(
            [nn.Linear(int(length/(i+1)), int(length/(i))) for i in range(num_layers-1, 0, -1)]
        )

    def encoder(self, x):
        for i, layer in enumerate(self.encoder_layer):
            x = layer(x)
            x = self.act_f(x) if i < len(self.encoder_layer) - 1 else x
        return x
    
    def decoder(self, x):
        for i, layer in enumerate(self.decoder_layer):
            x = layer(x)
            x = self.act_f(x) if i < len(self.decoder_layer) - 1 else x
        return x
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        x = Softmax_BA(num_cat, values, configspace, dim=1)(x)
        return x
    
    def give_latent_image(self, x):
        x = self.encoder(x)
        return x

In [ ]:
num_cat = []
for hp in configspace.get_hyperparameters():
    if type(hp) == CategoricalHyperparameter:
        num_cat.append(len(hp.choices))
    else:
        num_cat.append(False)

In [ ]:
def train(model, X_train, X_test, num_epochs, learning_rate, weight_decay=1e-5, plot_interval=10, verbose=False):
    
    loss_history = list()
    test_loss_history = list()
    # criterion = loss_function()
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=learning_rate, 
                                 weight_decay=weight_decay)
    
    for epoch in range(num_epochs):
        # Get a new batch of data, 64 key-value pairs in it
        ids = np.random.choice(X_train.shape[0], 64, replace=False)
        X_train = X_train[ids]
        
        # Convert to torch tensor, usually you also need to convert to float as in here.
        # X_train = torch.tensor(X_train).float()
        # X_test = torch.tensor(X_test).float()
     
        # Forward. Encodes and decodes and gives us the model's prediction.
        # model() actually calls 'forward()'
        output = model(X_train)
        output_test = model(X_test)

        # Calculate loss, defined above as mean squared error
        loss = loss_function(output, X_train, num_cat)
        loss_test = loss_function(output_test, X_test, num_cat)
        
        
        # === The backpropagation
        # Reset the gradients
        optimizer.zero_grad()
        # Calculate new gradients with backpropagation
        loss.backward()
        # Tune weights accoring to optimizer (it has the learnrate and weight decay as defined above)
        optimizer.step()
        
        
        # To do output stuff with loss and image, we have to detach() and convert back to numpy.
        loss = loss.detach().numpy()
        loss_test = loss_test.detach().numpy()
        
        # Append to loss history
        loss_history.append(loss)
        test_loss_history.append(loss_test)

        if verbose:
            print('Epoch: ' + str(epoch) + ". Train loss: " + str(loss.item()) + " Test loss: " + str(loss_test.item()))
        
        if epoch % plot_interval == 0 and epoch != 0:
            
            print('First 5x5 Dimension of prediction \n ')
            print(X_train[0, 22:31])
            print(output[0, 22:31])
            print("-"*100)
            
            """low_dim_train = model.give_latent_image(X_train)
            low_dim_test = model.give_latent_image(X_test)
            low_dim_train = low_dim_train.detach().numpy()
            low_dim_test = low_dim_test.detach().numpy()
            plt.scatter(low_dim_train[:, 0], low_dim_train[:, 1], s=10.0,label="train points")
            plt.scatter(low_dim_test[:, 0], low_dim_test[:, 1], s=10.0,label="test points")           
            
            plt.legend()
            plt.show()"""
    
    return loss_history, test_loss_history, model


def test(trained_model, X, num_plot):
    """ Test our autoencoder. """       
    for i in range(num_plot):
        """index = 0
        for cats in num_cat:
            if cats == False:
                index += 1
                continue
            plt.bar(np.arange(cats), X[i][index:index+cats], label="true", alpha=0.3)
            plt.bar(np.arange(cats), output[i][index:index+cats], label="prediction", alpha=0.3)
            plt.legend()
            plt.show()
            index += cats           
        print("last index true: " + str(X[i][-1])  + ", prediction: " + str(output[i][-1]))"""
        
        ids = np.random.choice(X.shape[0], 100)
        X = X[ids]
        
        X = torch.tensor(X).float()
        output = trained_model(X)
        
        loss = loss_function(output, X, num_cat)
        loss = loss.detach().numpy() 
        
        X = X.detach().numpy()
        output = output.detach().numpy()
        print("Input: \n %s \n Output: \n %s" % (X[:2, 15:25], output[:2, 15:25]))
        print("Train loss: " + str(loss.item()))
        print("-" * 10)
        

In [ ]:
import random
division = int(len(training.config_ids)* 0.75)

ids = np.arange(convert_data.shape[0])
np.random.shuffle(ids)
train_ids = ids[:division]
test_ids = ids[division:]

In [ ]:
def cross_entropy_one_hot(input, target):
    _, labels = target.max(dim=1)
    return nn.CrossEntropyLoss()(input, labels)

In [ ]:
def loss_function(label, predition, num_category):
    indexing = 0
    categorical_Loss = 0
    mse = nn.MSELoss()
    mse_Loss = 0
    
    for num, hp in enumerate(configspace.get_hyperparameters()):
        if type(hp) == CategoricalHyperparameter:
            confi_pred = predition[:, indexing:indexing+num_category[num]]
            confi_lable = label[:, indexing:indexing+num_category[num]]
            categorical_Loss += cross_entropy_one_hot(confi_lable, confi_pred)
            indexing += num_category[num]
        else:
            mse_Loss += mse(label[:, indexing], predition[:, indexing])
            indexing += 1
    
    #print("MSE: %s" % mse_Loss)
    #print("CE: %s" % categorical_Loss)
    
    return mse_Loss + categorical_Loss

In [ ]:
# New model
model = Autoencoder(convert_data.shape[1], "tanh", 3)

# Train the model and return loss history
loss_history, test_loss_history, new_model = train(model, 
                                        X_train=torch.tensor(convert_data[train_ids]).float(), 
                                        X_test=torch.tensor(convert_data[test_ids]).float(),
                                        num_epochs=1000, 
                                        learning_rate=1e-5, 
                                        weight_decay=1e-5, 
                                        plot_interval=100, 
                                        verbose=True)

# Plot the loss history. Careful: It's the train loss
plt.plot(loss_history, label="train")
plt.plot(test_loss_history, label="test")
plt.legend()
plt.show()

In [ ]:
# Print the test loss and plot some example images
test(new_model, convert_data_transform, num_plot=100)

In [ ]:
X = torch.tensor(convert_data).float()
Z = torch.tensor(convert_data_transform).float()
low_dim_rand = model.give_latent_image(X)
low_dim_rand = low_dim_rand.detach().numpy()
low_dim_local = model.give_latent_image(Z)
low_dim_local = low_dim_local.detach().numpy()

plt.scatter(low_dim_rand[:, 0], low_dim_rand[:, 1], s=10.0,label="random points")
plt.scatter(low_dim_local[:, 0], low_dim_local[:, 1], s=10.0,label="random points")
plt.show()

In [ ]:
from ConfigSpace.read_and_write import json

with open('./config_space.json', 'w') as f:
    f.write(json.write(configspace))

In [ ]:
X = torch.tensor(convert_data).float()
low_dim = model.give_latent_image(X)

low_dim = low_dim.detach().numpy()

In [ ]:
plt.scatter(low_dim[:, 0], low_dim[:, 1],)  # label="local points")

# plt.legend()
plt.show()

In [ ]:
def calculate_costvalue(dists, red_dists):
    """
    Helpfunction to calculate the costvalue to test how big the difference of distance is in the embedding
    and original space.

    Parameters
    ----------
    dists: np.array, shape(n_samples, n_samples)
        Matrix of the distances in the original space.
    red_dists: np.array, shape(n_samples, k_dimensions)
        Koordinates o

    Returns
    --------
    costvalue: float
        Costvalue of the distances of the two spaces.

        costvalues = sum_i sum_j=i+1 (distance_low_space_ij - distance_high_space_ij)
    """
    n_conf = dists.shape[0]
    low_dists = euclidean_distances(red_dists)
    costvalue = []
    mean_actual = []

    for i in range(n_conf - 1):
        for j in range(i+1, n_conf):
            costvalue.append((dists[i][j] - low_dists[i][j])**2)
            mean_actual.append(low_dists[i][j])

    mean_actual_value = sum(mean_actual) / len(mean_actual)
    actual = [(mean_actual_value - dif)**2 for dif in mean_actual]
    pred_actual = sum(costvalue)
    rse = pred_actual / sum(actual)


    costvalue = sum(costvalue) / len(costvalue)
    print("costvalue")
    print(costvalue)
    print("rse")
    print(rse)
    return costvalue

In [ ]:
# Softmax

m = nn.Softmax(dim=1)
test = torch.randn(2, 3)
output = m(test)

print(test)
print(output)

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(4, 1, requires_grad=True)
target = torch.empty(4, dtype=torch.long).random_(1)
output = loss(input, target)
output.backward()

In [ ]:
input

In [ ]:
torch.empty(4, dtype=torch.long).random_(1)

In [ ]:
image = output.detach().numpy()
# image = image[0].reshape(image.shape[1])
plt.imshow(image)

# Misc

One-hot-encoder version with -1 for each one-hot dimension &rarr; nan by categorical with 4 choices is [-1, -1, -1, -1]

In [ ]:
# one hot encoding
def one_hot_encode(config):
    # Code from PhMueller
    
    # Create array with one hot encoded values
    result_vec = np.array([]).reshape((-1, 1))  # .astype(object)

    for i, name in enumerate(configspace.get_hyperparameter_names()):
        
        val = np.array(config.get(name)).reshape((-1, 1))
        
        # Case if this value is not given in the configuration
        if val == [[None]]:
            # Test, maybe this is not working  
            if len(result_vec) == 0 and type(configspace.get_hyperparameter(name)) == CategoricalHyperparameter:
                cats = len(configspace.get_hyperparameters()[i].choices)
                result_vec = np.array([-1] * cats).reshape((1, len(np.array([-1] * cats))))
            elif len(result_vec) == 0 and type(configspace.get_hyperparameter(name)) != CategoricalHyperparameter:
                result_vec = np.array([-1]).reshape((-1, 1))
            elif len(result_vec) > 0 and type(configspace.get_hyperparameter(name)) == CategoricalHyperparameter:
                cats = len(configspace.get_hyperparameters()[i].choices)
                result_vec = np.hstack((result_vec, np.array([-1] * cats).reshape((1, len(np.array([-1] * cats))))))
            else:
                result_vec = np.hstack((result_vec, [[-1]]))
        
        # case if categorical
        elif type(values[i]) is OneHotEncoder:
            if len(result_vec) == 0:
                result_vec = values[i].transform(val).toarray()  # .astype(object)
            else:
                result_vec = np.hstack((result_vec, 
                                        values[i].transform(val).toarray()))

        # if it is one
        else:
            if len(result_vec) == 0:
                result_vec = values[i].transform(val)
            else:
                result_vec = np.hstack((result_vec, 
                                        values[i].transform(val)))
    return result_vec

In [15]:
oe = OneHotEncoder(categories='auto').fit(np.array([1,2,'-1']).reshape((-1,1)))
oe.categories_

oe.transform(np.array(1).reshape((-1, 1))).toarray()

/home/abj/anaconda3/envs/visHyp/lib/python3.5/site-packages/numpy/lib/arraysetops.py:518: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/home/abj/anaconda3/envs/visHyp/lib/python3.5/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ValueError: Found unknown categories [1] in column 0 during transform